# Maximización de los valores de expectación / EM

Es un proceso iterativo utilizado para encontrar el mejor valor de expectación. Se parte de una suposición inicial de los centroides de las clases y mediane un proceso iterativo que busca maximizar la probabilidad de los puntos a pertencer a una u otra clase. 

El algoritmo EM se utiliza para encontrar maximos locales de probabilidad cuando los modelos contienen variables inferidas o [variables latentes](https://es.wikipedia.org/wiki/Variable_latente), así como datos y parámetos faltantes. El modelo parte de asumir que a cada punto de los datos conocidos le corresponde uno faltante. En partícular se busca maximizar la [probabilidad a posteriori](https://es.wikipedia.org/wiki/Probabilidad_a_posteriori) de que un punto pertenezca a una clase.
La siguiente animación tomada de [Wikipedia](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm#/media/File:EM_Clustering_of_Old_Faithful_data.gif) muestra la convergencia del modelo.

Volviendo al ejemplo sobre las etiquetas de genero de la sección anterior, esta información son los datos faltantes que se mencionan aquí, los cuales corresponden con las etiquetas o clases del archivo de datos. 

EM asigna los puntos que deseamos clasificar a una clase en base a una probabilidad.

El algoritmo EM funciona iterativamente aplicando dos pasos:

1. **Expectación** (E): Assignar los puntos al cluster más cercano.
* **Maximización** (M): Calcular la media de las nuevas distribuciones de cada clase.

En el paso E se actualizan los valores de expectación mediane la función de probabilidad condicional de que los puntos pertenezcan a una clase:

$$Q(\theta|\theta^{(t)}) = E_{Z|X,\theta^{(t)}}[logL(\theta,X,Z]$$

En donde $X$ son los datos de observación, $Z$ son los valores falantes y $\theta$ un vector de los parámetros conocidos.

Y posteriormente se busca maximizar la expresión anterior con respecto a los parámetros $\theta$.

Si se aprender más sobre el algoritmo, fundamentación y derivación matemática, se recomiendan los siguientes artículos:

[Frank Dellaert, The Expectation Maximization Algorithm, 2002](https://www.cc.gatech.edu/~dellaert/em-paper.pdf).

[Sean Borman, The Expectation Maximization Algorithm A short tutorial](https://www.seanborman.com/publications/EM_algorithm.pdf)

[Chuong B Do & Serafim Batzoglou, What is the expectation maximization algorithm?, 2008](https://www.nature.com/articles/nbt1406)

## EM en Python

Para ejemplificar el uso de EM en Python, se utilizará el famoso ejemplo del _experimento del volado con una moneda_. Para ello se utiliza el siguiente artículo de _Nature_ publicado en _Computational Biology_: [Chuong B Do & Serafim Batzoglou](http://ai.stanford.edu/~chuongdo/papers/em_tutorial.pdf)

Primeramente imagina que tienes dos monedas, $A$ y $B$, de sesgos desconocidos, $\theta_A$ y $\theta_B$ respectivamente. Las proabilidades son entonces $\theta_A$ la probabilidad de obtener cara y $1-\theta_A$ de obtener cruz, para la moneda $A$, y similar para la moneda $B$. Nuestro objetivo es entonces conocer el sesgo $\theta = (\theta_A,\theta_B)$. Para ello se repetirán los siguientes pasos 5 veces:

1. Escoger una moneda al azar (con igual probabilidad).
2. Realizar 10 lanzamientos con la moneda elegida.

Durante el experimento guardamos el número de caras del $i$-ésimo lanzamiento, $x_i$ y una etiqueta que identifique a dicha moneda, $z_i$. Buscamos entonces conocer el máximo valor de expectación para cada moneda.

Podemos estimar $\theta_A$ y $\theta_B$ de la siguiente manera:

$$\hat{\theta}_A = \frac{n_A}{N_A}$$

y

$$\hat{\theta}_B = \frac{n_B}{N_B}$$

en donde $n_A$ es el número de caras obtenidas utilizando la moneada A y $N_A$ es el número total de lanzamientos; similar para la moneda B.

Ahora, $log P(x,z;\theta)$ es la probabilidad de obtener cualquiera de los vectores en partícular, de la cuenta del número de caras $x$ de la moneza $z$, de manera que las expresiones anteriores resolverían el problema de manera muy simple.

Ésto se parece, en nuestro ejemplo de introducción, al caso en el cual se tiene un archivo de datos completo; se conoce a qué moneda le pertenece cada conjunto de datos de los lanzamientos, así como se conocían las etiquetas del genero en nuestro ejemplo.

Ahora imaginate que sólo conocemos la información del número de caras obtenidas, pero no su identidad de a qué moneda le pertenece cada lanzamiento de dicha información. Entonces la información faltante, $Z$, son si los lanzamientos pertenecen a  la moneda A o B. 

El objetivo es determinar, para cada uno de los cinco experimentos, si los lanzamientos se hicieron utilizando la moneda A o la moneda B.

!TERMINAR EL EJEMPLO¡

# Mezclas Gausianas